In [3]:
import pandas as pd
import re
import string

In [4]:
# from pathlib import Path
# GDRIVE_PATH = '/content/gdrive/My Drive/code/doc_ua/'

# if not Path(GDRIVE_PATH).exists():
#     from google.colab import drive
#     drive.mount('/content/gdrive')

In [5]:
body = pd.read_csv('./Doc.ua/body.csv')

In [6]:
body.head()

,id,parent,zone_id,name,active,deleted
0,1,NaN,"1,1__1",Голова,1,0
1,2,1.0,"1_5_1,1_5",Нос,1,0
2,3,NaN,"4,1_11",Живот,1,0
3,4,3.0,NaN,печень,1,1
4,5,NaN,NaN,Общие симптомы,1,0


In [7]:
body.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       54 non-null     int64  
 1   parent   44 non-null     float64
 2   zone_id  50 non-null     object 
 3   name     54 non-null     object 
 4   active   54 non-null     int64  
 5   deleted  54 non-null     int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 2.7+ KB


In [10]:
body_symptom = pd.read_csv('./Doc.ua/body_symptom.csv')

In [11]:
body_symptom.head()

,id,body_id,symptom_id
0,1,16,14
1,2,16,220
2,3,48,3
3,6,30,3
4,7,30,4


In [12]:
body_symptom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id          1497 non-null   int64
 1   body_id     1497 non-null   int64
 2   symptom_id  1497 non-null   int64
dtypes: int64(3)
memory usage: 35.2 KB


In [13]:
import csv
def _read_csv(fp):
    with open(fp, newline='', encoding='utf-8') as f:
        df = []
 
        reader = csv.reader(f, doublequote=False, escapechar='\\',
                            delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            df.append(row)
 
        return pd.DataFrame(df)

In [14]:
disease = _read_csv('./Doc.ua/disease.csv')
disease = disease.rename(columns=disease.iloc[0]).drop(disease.index[0])

In [15]:
disease.head()

,id,group_id,name,alias,gender,deleted,description,about,popularity,active
1,4,5,Анафилактический шок,anafilakticheskij-shok,all,1,<p><strong>Анафилактический шок (Анафилаксия)<...,<h2>Симптомы</h2>\r\n<p>Симптомы анафилактичес...,0,1
2,5,5,Бронхиальная астма,bronhialnaya-astma,all,0,<p><strong>Бронхиальная астма (БА) </strong>&n...,<p>Бронхиальная астма была признана наиболее ч...,0,1
3,6,5,Крапивница,krapivnica,all,0,<p><strong>Крапивница</strong> &mdash; болезнь...,<p>Крапивница занимает 2 место после бронхиаль...,0,1
4,7,5,Отек Квинке,otek-kvinke,all,0,<p><strong>Ангионевротический отек (отек Квинк...,<h2>Причины</h2>\r\n<p>В 80% случаев причиной ...,0,1
5,8,5,Поллиноз (Сенная лихорадка),pollinoz,all,0,<p><strong>Поллиноз</strong> (сезонный насморк...,<p>Аллергия у взрослых и детей &ndash; гиперчу...,0,1


In [16]:
disease['description'][1]

'<p><strong>Анафилактический шок (Анафилаксия)</strong> относится к опасным ситуациям, встречающимся в человеческой жизни, которые угрожают непосредственно жизнедеятельности человека. Анафилактический шок выражается повышенной чувствительностью к аллергену или аллергической реакцией.</p>\r\n<p>Зачастую анафилаксия развивается вследствие повторного контакта с аллергеном. Например, если наблюдается чувствительность к яду или слюне пчел, то первый укус может пройти без последствий, а дальнейший способен развивать анафилактическую реакцию, которая может проявиться как через полчаса, так и через 12 часов.</p>'

# Cleaning html text
Мне кажется, этот вариант лучше

In [17]:
from bs4 import BeautifulSoup
import re

In [18]:
def clean_html(html):
    #убираем точки, они мешают работать BeautifulSoup
    text = re.sub(r'\.', ' ', html)
    soup = BeautifulSoup(html)
    text = soup.get_text()  
    #убираем \n|\r|\xa0
    text = re.sub(r'\n|\r|\xa0|mdash|nbsp|ndash|igeo|b', ' ', text)
    text = re.sub(r'[^\w ]', '', text)
#     text = re.sub(r'\\', ' ', text)
#     text = re.sub(r'"', ' ', text)
    #убираем лишние пробелы
    text = re.sub(r' +', ' ', text) 
    return text.strip()

In [19]:
ex2 = '\\бульканье\\" в грудной клетке"'
clean_html(ex2)

'бульканье в грудной клетке'

In [20]:
disease['description'] = disease['description'].apply(clean_html)
disease['about'] = disease['about'].apply(clean_html)

In [21]:
disease.head()

,id,group_id,name,alias,gender,deleted,description,about,popularity,active
1,4,5,Анафилактический шок,anafilakticheskij-shok,all,1,Анафилактический шок Анафилаксия относится к о...,Симптомы Симптомы анафилактического шока голов...,0,1
2,5,5,Бронхиальная астма,bronhialnaya-astma,all,0,Бронхиальная астма БА заболевание для которого...,Бронхиальная астма была признана наиболее част...,0,1
3,6,5,Крапивница,krapivnica,all,0,Крапивница болезнь аллергического происхождени...,Крапивница занимает 2 место после бронхиальной...,0,1
4,7,5,Отек Квинке,otek-kvinke,all,0,Ангионевротический отек отек Квинке острая алл...,Причины В 80 случаев причиной развития отека К...,0,1
5,8,5,Поллиноз (Сенная лихорадка),pollinoz,all,0,Поллиноз сезонный насморк сенная лихорадка одн...,Аллергия у взрослых и детей гиперчувствительна...,0,1


In [22]:
# import re

# html = '''<p>Зачастую анафилаксия развивается вследствие повторного контакта с аллергеном. Например, если наблюдается чувствительность к яду или слюне пчел, то первый укус может пройти без последствий, а дальнейший способен развивать анафилактическую реакцию, которая может проявиться как через полчаса, так и через 12 часов.</p>","<h2>Симптомы</h2>'''

# re.sub("<.*?>", " ", html)

In [23]:
# nohtml_6 = []
# for s in range(len(disease[6])):
#     nohtml_6.append(re.sub("<.*?>", " ", disease[6][s]))

In [24]:
# disease['desc'] = nohtml_6

In [25]:
# почистити від &ndash, \r, \n, ...
# disease

In [27]:
disease_body_symptom = pd.read_csv('./Doc.ua/disease_body_symptom.csv')

In [28]:
disease_body_symptom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5625 entries, 0 to 5624
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   body_symptom_id  5625 non-null   int64
 1   disease_id       5625 non-null   int64
dtypes: int64(2)
memory usage: 88.0 KB


In [29]:
disease_body_symptom.head()

,body_symptom_id,disease_id
0,1,71
1,1,87
2,1,93
3,1,118
4,1,119


In [31]:
disease_symptom = pd.read_csv('./Doc.ua/disease_symptom.csv')

In [32]:
disease_symptom.shape

(3519, 2)

In [33]:
disease_symptom.head()

,disease_id,symptom_id
0,4,5
1,4,8
2,4,52
3,4,56
4,4,68


In [36]:
doc_spec = pd.read_csv("./Doc.ua/doc_spec.csv")

In [37]:
doc_spec.head()

,specialty_id,doctor_id,id,degree_id,category_id,is_child_doctor,active,deleted,since,default
0,3,14,16,NaN,NaN,NaN,1.0,0.0,NaN,0
1,48,15,17,NaN,NaN,NaN,1.0,0.0,NaN,0
2,13,14,19,NaN,NaN,NaN,1.0,0.0,NaN,0
3,44,14,20,NaN,NaN,NaN,1.0,0.0,NaN,0
4,8,16,21,NaN,NaN,NaN,1.0,0.0,NaN,0


In [38]:
doc_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41082 entries, 0 to 41081
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   specialty_id     41082 non-null  int64  
 1   doctor_id        41082 non-null  int64  
 2   id               41082 non-null  int64  
 3   degree_id        27451 non-null  float64
 4   category_id      27451 non-null  float64
 5   is_child_doctor  27234 non-null  float64
 6   active           41080 non-null  float64
 7   deleted          40919 non-null  float64
 8   since            14977 non-null  float64
 9   default          41082 non-null  int64  
dtypes: float64(6), int64(4)
memory usage: 3.1 MB


In [40]:
doctor = _read_csv('./Doc.ua/doctor.csv')

In [41]:
# колонки з NULL?
doctor.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,id,alias,category,experience,to_home,price,price_from,user_id,experience_description,short_about,...,experience_update_on,last_name_uk,name_uk,second_name_uk,about_uk,manager_id,operational_status,duty_doconline,nin,event_ref_id
1,12,6961c636ef541a9edcc4b2b1b1b82299,4,14,1,NULL,NULL,21,NULL,,...,NULL,Анонімус,Р,В,,498696,1,0,0,0
2,13,835b0c353cb19e85e3a1942da861aa20,3,16,NULL,NULL,NULL,23,NULL,,...,NULL,Анонімус,Д,Ю,,NULL,NULL,0,0,0
3,14,e552562ec79bc3be94d71fd444bf4de9,4,21,NULL,NULL,NULL,24,NULL,,...,NULL,Анонімус,Е,Д,,NULL,NULL,0,0,0
4,15,a311038a0040d39158e7fda962864cc2,1,8,NULL,NULL,NULL,26,NULL,,...,NULL,Анонімус,А,М,,NULL,NULL,0,0,0


In [42]:
doctor = doctor.rename(columns=doctor.iloc[0]).drop(doctor.index[0])

In [43]:
doctor.columns

Index(['id', 'alias', 'category', 'experience', 'to_home', 'price',
       'price_from', 'user_id', 'experience_description', 'short_about',
       'about', 'deleted', 'academic_title', 'send_request_phone',
       'can_edit_self', 'courses', 'association', 'procedures', 'rate',
       'review_count', 'active', 'media_id', 'education', 'additional_phone',
       'price_description', 'child', 'recommend_review_count', 'adult',
       'work_specialties', 'info', 'promo', 'without_contract', 'video',
       'no_order', 'pet', 'license', 'integration_rate', 'duty', 'ref_url',
       'moderated', 'status', 'terms_and_conditions_accepted',
       'privacy_policy_accepted', 'region_id', 'city_id', 'avatar_img',
       'ref_prompt_ua', 'ref_prompt', 'is_passport', 'passport_status',
       'diploma_status', 'tax_status', 'is_diploma_old', 'dynamic_link',
       'ref_doctor_id', 'experience_update_on', 'last_name_uk', 'name_uk',
       'second_name_uk', 'about_uk', 'manager_id', 'operational_st

In [44]:
doctor.isnull().sum()

id                    0
alias                 0
category              0
experience            0
to_home               0
                     ..
manager_id            0
operational_status    0
duty_doconline        0
nin                   0
event_ref_id          0
Length: 65, dtype: int64

In [46]:
doctor_diseases = pd.read_csv('./Doc.ua/doctor_diseases.csv')

In [47]:
doctor_diseases.head()

,doc_id,dis_id
0,13,13
1,13,14
2,13,15
3,13,17
4,13,21


In [49]:
hackathon_order = _read_csv('./Doc.ua/hackathon_order.csv')
hackathon_order = hackathon_order.rename(columns=hackathon_order.iloc[0]).drop(hackathon_order.index[0])
hackathon_order.drop(hackathon_order.columns[70:], axis=1, inplace=True)
hackathon_order['comment'] = hackathon_order['comment'].apply(clean_html)

/home/ghadd/.local/lib/python3.9/site-packages/bs4/__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/home/ghadd/.local/lib/python3.9/site-packages/bs4/__init__.py:332: MarkupResemblesLocatorWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [50]:
hackathon_order.head()

,id,adult,email,last_name,second_name,name,gender,date_create,date_update,comment,...,test,loyalty,slots_shown,integration_status_id,price_marker,quantity_marker,another_person,referral,status_changed_at,phone_hash
1,54,1,test@test.com,Аннонимус,В,О,female,2014-02-10 07:16:04,2021-03-18 08:59:10,Прием к врачу Попович Срок беременности 15 нед...,...,0,0,0,NULL,NULL,NULL,0,NULL,NULL,bb41ca8ba3ef119e5e87d34cb8fc586e
2,59,1,test@test.com,Аннонимус,NULL,Ц,female,2014-02-12 02:12:23,2021-03-18 08:59:10,здраствуйте хотелабы у вас проконсультироваца ...,...,0,0,0,NULL,NULL,NULL,0,NULL,NULL,1d2d2d3b7c25ae8cd025ba16b7840e79
3,62,0,test@test.com,Аннонимус,NULL,NULL,male,2014-02-12 08:11:32,2021-03-12 20:54:49,Ребенку поставили диагноз рефлюкс 23 степени,...,0,0,0,NULL,NULL,NULL,0,NULL,NULL,1f179ea8ac16342dd5adb96ab74b4ee5
4,64,1,test@test.com,Аннонимус,NULL,А,male,2014-02-12 10:39:22,2021-03-18 08:59:10,Кровотечение слабость,...,0,0,0,NULL,NULL,NULL,0,NULL,NULL,32c48af4afcfd0b2ef5b5578e7187f64
5,65,1,test@test.com,Аннонимус,,И,male,2014-02-12 22:43:41,2021-03-18 08:59:10,Грибок на коже консультация и лечение,...,0,0,0,NULL,NULL,NULL,0,NULL,NULL,0b4ebd24e8845d8d31e696b244d1112d


In [51]:
# колонки email, last_name, ... видалити (?)
hackathon_order.drop(columns = ['email', 'last_name','second_name', 'name'], inplace=True)
# comment,  симптоми?

In [52]:
hackathon_order.mainStatus.unique()

array(['complete', 'fail', 'new', 'NULL', '21385', '', '0', '1', '3',
       '114', '42', '4', '6', '20', '27', '9951', '24', 'approve',
       'onApprove'], dtype=object)

In [53]:
hackathon_order[hackathon_order['mainStatus'] == '24']

,id,adult,gender,date_create,date_update,comment,deleted,address,operator,specialty_id,...,test,loyalty,slots_shown,integration_status_id,price_marker,quantity_marker,another_person,referral,status_changed_at,phone_hash
51677,",0,г.Аннонимный улица Неизвестная""""",422475,605,NULL,1,4241,NULL,NULL,complete,,...,,,,,,,,,,


In [54]:
len(pd.unique(hackathon_order['doctor_id']))

7087

In [55]:
comments = hackathon_order[['comment', 'doctor_id']]

In [56]:
comments

,comment,doctor_id
1,Прием к врачу Попович Срок беременности 15 нед...,36
2,здраствуйте хотелабы у вас проконсультироваца ...,36
3,Ребенку поставили диагноз рефлюкс 23 степени,65
4,Кровотечение слабость,152
5,Грибок на коже консультация и лечение,243
...,...,...
62094,Зробити узі мочевого міхура простати і передмі...,9881
62095,Здравствуйте Сдавал кровь на анализы для карди...,4148
62096,ежегодное обследование,2327
62097,Нежить температура та головний біль,10469


In [57]:
doctor.columns

Index(['id', 'alias', 'category', 'experience', 'to_home', 'price',
       'price_from', 'user_id', 'experience_description', 'short_about',
       'about', 'deleted', 'academic_title', 'send_request_phone',
       'can_edit_self', 'courses', 'association', 'procedures', 'rate',
       'review_count', 'active', 'media_id', 'education', 'additional_phone',
       'price_description', 'child', 'recommend_review_count', 'adult',
       'work_specialties', 'info', 'promo', 'without_contract', 'video',
       'no_order', 'pet', 'license', 'integration_rate', 'duty', 'ref_url',
       'moderated', 'status', 'terms_and_conditions_accepted',
       'privacy_policy_accepted', 'region_id', 'city_id', 'avatar_img',
       'ref_prompt_ua', 'ref_prompt', 'is_passport', 'passport_status',
       'diploma_status', 'tax_status', 'is_diploma_old', 'dynamic_link',
       'ref_doctor_id', 'experience_update_on', 'last_name_uk', 'name_uk',
       'second_name_uk', 'about_uk', 'manager_id', 'operational_st

In [61]:
doctor = doctor.rename(columns={'id': 'doctor_id'})
doctor

,doctor_id,alias,category,experience,to_home,price,price_from,user_id,experience_description,short_about,...,experience_update_on,last_name_uk,name_uk,second_name_uk,about_uk,manager_id,operational_status,duty_doconline,nin,event_ref_id
1,12,6961c636ef541a9edcc4b2b1b1b82299,4,14,1,NULL,NULL,21,NULL,,...,NULL,Анонімус,Р,В,,498696,1,0,0,0
2,13,835b0c353cb19e85e3a1942da861aa20,3,16,NULL,NULL,NULL,23,NULL,,...,NULL,Анонімус,Д,Ю,,NULL,NULL,0,0,0
3,14,e552562ec79bc3be94d71fd444bf4de9,4,21,NULL,NULL,NULL,24,NULL,,...,NULL,Анонімус,Е,Д,,NULL,NULL,0,0,0
4,15,a311038a0040d39158e7fda962864cc2,1,8,NULL,NULL,NULL,26,NULL,,...,NULL,Анонімус,А,М,,NULL,NULL,0,0,0
5,16,738b6f1f22e406fe665d9a059af506a6,4,20,NULL,NULL,NULL,29,NULL,,...,NULL,Анонімус,М,І,,NULL,NULL,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34432,34982,NULL,4,22,NULL,NULL,NULL,849847,NULL,,...,NULL,Анонімус,NULL,NULL,,NULL,NULL,0,0,0
34433,34983,NULL,2,15,NULL,NULL,NULL,849850,NULL,,...,NULL,Анонімус,NULL,NULL,,NULL,NULL,0,0,0
34434,34984,NULL,3,14,NULL,NULL,NULL,849869,NULL,,...,NULL,Анонімус,NULL,NULL,,NULL,NULL,0,0,0
34435,34985,NULL,3,18,NULL,NULL,NULL,849876,NULL,,...,NULL,Анонімус,NULL,NULL,,NULL,NULL,0,0,0


In [62]:
comments_doctor = comments.merge(doctor['doctor_id'], on='doctor_id', how='inner')

In [64]:
import spacy
nlp = spacy.load("ru_core_news_lg")

In [65]:
def spacy_tokinize(sentence):
    template = nlp(sentence)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    for token in template:
        lexeme = nlp.vocab[token.text]
        if lexeme.is_stop == False:
            filtered_sentence.append(token) 
    return  filtered_sentence 

In [66]:
def leave_meaningful(s):
    meaningful = []

    for token in s:
        if token.pos_ in ["NOUN", "VERB"]:
            meaningful.append(token.lemma_)

    meaningful = np.unique(np.array(meaningful))
    return meaningful

In [ ]:
comments_doctor.comment = comments_doctor.comment.apply(spacy_tokinize)

In [ ]:
comments_doctor.comment = comments_doctor.comment.apply(leave_meaningful)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from string import punctuation
punctuation += '—'

In [ ]:
stop = set(stopwords.words('russian'))
punct = list(punctuation)

In [ ]:
import sys
import re
import os

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
import string

import sentencepiece as spm

class Preprocessor():

    def __init__(self, language = 'russian'):
        self.language = language
        if instream:
            self.corpus_orig = self.read_string(instream)
        else:
            self.corpus_orig = self.read_articles(sys.stdin)
        self.corpus = self.preprocess(self.corpus_orig)

    def preprocess(self, corpus_list):
        preproc_corpus_list = []
        stopset = stopwords.words(self.language) + list(string.punctuation)
        for corpus in corpus_list:
            corpus = corpus.lower()
            corpus = " ".join([ i for i in word_tokenize(corpus) if i not in stopset])
            preproc_corpus_list.append(corpus)
        return preproc_corpus_list

    def preprocess_sp(self, corpus_list):
        return self.sentencepiece_apply(self.preprocess(corpus_list))

    def sentencepiece_train(self, corpus_list, vocab_size = 24000):
        fp_out = open("./sp_corpus.txt", 'w')
        for corpus in corpus_list:
            print(corpus, file=fp_out)
        fp_out.close()
        spm.SentencePieceTrainer.Train(f"--input=sp_corpus.txt --model_prefix=sp_model --vocab_size={vocab_size} --max_sentence_length=1000 --character_coverage=1.0 --num_threads=4 --hard_vocab_limit=false")
        return None

    def sentencepiece_apply(self, corpus_list):
        sent_corpus_list = []
        sp = spm.SentencePieceProcessor()
        sp.Load("./sp_model.model")
        for corpus in corpus_list:
            sent_corpus_list.append(" ".join(sp.EncodeAsPieces(corpus)))
        return sent_corpus_list

    def read_articles(self, fp):
        articles = []
        for line in fp:
            if line.strip() != "":
                articles.append(line.strip())
        return articles

    def read_string(self, mystring):
        articles = [mystring]
        return articles


def main():
    vocab_size = 24000
    if len(sys.argv) > 1:
        vocab_size = sys.argv[1]
    print("Create Preprocessor")
    preprocessor = Preprocessor(language = 'english')
    print("Train spm")
    preprocessor.sentencepiece_train(preprocessor.corpus, vocab_size = vocab_size)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'sentencepiece'

In [ ]:
stopset = stopwords.words(se) + list(string.punctuation)
def preprocess(text):
    lst = word_tokenize(text)
    lst = [word for word in lst if word not in stop and word not in punct]
    return lst

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/qgelena/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
hackathon_order['comment'][8]

NameError: name 'hackathon_order' is not defined

In [ ]:
hackathon_order.shape

(62098, 65)

In [ ]:
hackathon_order.isnull().sum()

adult                0
gender               0
date_create          0
date_update          0
comment              0
                    ..
quantity_marker      0
another_person       0
referral             0
status_changed_at    0
phone_hash           0
Length: 65, dtype: int64

In [ ]:
hackathon_order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62098 entries, 1 to 62098
Data columns (total 65 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   adult                        62098 non-null  object
 1   gender                       62098 non-null  object
 2   date_create                  62098 non-null  object
 3   date_update                  62098 non-null  object
 4   comment                      62098 non-null  object
 5   deleted                      62098 non-null  object
 6   address                      62098 non-null  object
 7   operator                     62098 non-null  object
 8   specialty_id                 62098 non-null  object
 9   user_id                      62098 non-null  object
 10  order_status_id              62098 non-null  object
 11  doctor_id                    62098 non-null  object
 12  affiliate_id                 62098 non-null  object
 13  timetable_id                 62

In [ ]:
specialty = _read_csv('./Doc.ua/specialty.csv')
specialty = specialty.rename(columns=specialty.iloc[0]).drop(specialty.index[0])
specialty['content'] = specialty['content'].apply(clean_html)

/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
specialty.head()

,id,name,alias,seo_title,keywords,description,content,date_create,date_update,deleted,case_parental,case_parental_plural,case_plural,top,popular,child,to_home,child_doctor,to_home_doctor,name_local
1,1,Окулист (офтальмолог),okulist,NULL,NULL,NULL,"Окулист или офтальмолог - врач, занимающийся и...",NULL,NULL,1,NULL,NULL,NULL,NULL,1,1,1,NULL,NULL,NULL
2,2,Терапевт,terapevt,NULL,NULL,NULL,Терапевт - лечит пациентов,NULL,NULL,1,терапевта,терапевтов,терапевты,NULL,1,1,1,NULL,NULL,NULL
3,3,Акушер-гинеколог,akusher-ginekolog,NULL,NULL,NULL,"Акушер-гинеколог – врач узкой специализации, з...",NULL,NULL,0,Акушера-гинеколога,Акушеров-гинекологов,Акушеры-гинекологи,1,0,0,1,,,Акушер-гінеколог
4,4,Аллерголог,allergolog,NULL,NULL,NULL,"Аллерголог — специалист, занимающийся диагност...",NULL,NULL,0,Аллерголога,Аллергологов,Аллергологи,NULL,0,1,1,"<p style=""text-align: justify;"">Детский аллерг...",,Алерголог
5,5,Андролог,androlog,NULL,NULL,NULL,"Андролог — врач, решающий проблемы сексуальной...",NULL,NULL,0,Андролога,Андрологов,Андрологи,NULL,0,1,1,"<p style=""text-align: justify;"">Детский андрол...",,Андролог


In [ ]:
specialty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 1 to 128
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    128 non-null    object
 1   name                  128 non-null    object
 2   alias                 128 non-null    object
 3   seo_title             128 non-null    object
 4   keywords              128 non-null    object
 5   description           128 non-null    object
 6   content               128 non-null    object
 7   date_create           128 non-null    object
 8   date_update           128 non-null    object
 9   deleted               128 non-null    object
 10  case_parental         128 non-null    object
 11  case_parental_plural  128 non-null    object
 12  case_plural           128 non-null    object
 13  top                   128 non-null    object
 14  popular               128 non-null    object
 15  child                 128 non-null    ob

In [ ]:
symptoms = pd.read_csv('./Doc.ua/symptom.csv')
symptoms['name'] = symptoms['name'].apply(clean_html)

In [ ]:
symptoms.head()

,id,name,body_id,deleted,gender
0,1,Боль в горле,NaN,1,all
1,2,боль,4.0,1,all
2,3,Тошнота,98.0,0,all
3,4,Тяжесть в грудной клетке,26.0,1,all
4,5,Удушье,98.0,0,all


In [ ]:
symptoms.shape, len(pd.unique(symptoms.name))
#то есть почти все уникальны и их надо обрабатывать

((937, 5), 920)

In [ ]:
all_symptoms = symptoms['name'].str.cat(sep=' ').lower()
stopset = stopwords.words('english') + list(string.punctuation)

In [ ]:
words = [w for w in set(word_tokenize(all_symptoms)) if (len(w) >= 3 and w not in stopset)]

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
def symptoms_preprocess(words):
    pymorphy_res = [morph.parse(x) for x in words]
    r = [res[0].normal_form for res in pymorphy_res]
    return r

In [ ]:
symptom_words = symptoms_preprocess(words)

In [ ]:
symptom_words

['холодный',
 'уменьшение',
 'сухой',
 'стрелять',
 'речь',
 'спазм',
 'повышенный',
 'температура',
 'выведение',
 'шаг',
 'подёргивание',
 'нетипичный',
 'железа',
 'ход',
 'травма',
 'щель',
 'менструальный',
 'капля',
 'окружность',
 'худощавый',
 'мочеиспускание',
 'опистотонус',
 'холод',
 'рост',
 'кондилома',
 'запрокидывание',
 'разрушение',
 'зуд',
 'зрение',
 'болезненный',
 'сужение',
 'сокращение',
 'анорексия',
 'слизистый',
 'падение',
 'впадина',
 'залысина',
 'менструальный',
 'багрово-красный',
 'нарушение',
 'незаращение',
 'пот',
 'прекращение',
 'забывчивость',
 'тембр',
 'укачивание',
 'зубной',
 'мочить',
 'сумеречный',
 'слабость',
 'брюшной',
 'ограничение',
 'язык',
 'грудь',
 'зависимость',
 'желе',
 'утруднить',
 'клетка',
 'врождённый',
 'зуб',
 'изжога',
 'красно-багровый',
 'диарея',
 'подреберье',
 'задний',
 'глазной',
 'потрескаться',
 'паралич',
 'припухлость',
 'условие',
 'отвращение',
 'сонный',
 'красный',
 'гнусавость',
 'характер',
 'пах',
 'маз

In [ ]:
Xy = pd.read_csv('./new_xy.csv')

In [ ]:
Xy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  782 non-null    int64 
 1   X           782 non-null    object
 2   y           782 non-null    object
dtypes: int64(1), object(2)
memory usage: 18.5+ KB


In [ ]:
!pip3 install plotly
!pip3 install --upgrade nbformat
!pip3 install nltk
!pip3 install spacy # spaCy is an open-source software library for advanced natural language processing
!pip3 install WordCloud
!pip3 install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

Requirement already up-to-date: nbformat in /usr/local/lib/python3.7/dist-packages (5.1.2)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# download stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Obtain additional stopwords from nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [ ]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [ ]:
comment2doc_id = pd.read_csv(GDRIVE_PATH+'comment2doc_id.csv')

In [ ]:
# Apply the function to the dataframe
comments['clean'] = comments['comment'].apply(preprocess)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
comments.head()

,comment,doctor_id,clean
1,Прием к врачу Попович Срок беременности 15 нед...,36,"[прием, врачу, попович, срок, беременности, не..."
2,здраствуйте хотелабы у вас проконсультироваца ...,36,"[здраствуйте, хотелабы, проблема, большого, ха..."
3,Ребенку поставили диагноз рефлюкс 23 степени,65,"[ребенку, поставили, диагноз, рефлюкс, степени]"
4,Кровотечение слабость,152,"[кровотечение, слабость]"
5,Грибок на коже консультация и лечение,243,"[грибок, коже, консультация, лечение]"


In [ ]:
# Obtain the total words present in the dataset
list_of_words = []
for i in comments.clean:
    for j in i:
        list_of_words.append(j)

In [ ]:
list_of_words

['прием',
 'врачу',
 'попович',
 'срок',
 'беременности',
 'недель',
 'запись',
 'сдачу',
 'анализов',
 'желаемое',
 'время',
 'приема',
 'утро',
 'здраствуйте',
 'хотелабы',
 'проблема',
 'большого',
 'характера',
 'баюсь',
 'ошыбаюсь',
 'решыла',
 'мужем',
 'записаца',
 'прием',
 'проблема',
 'кандидоз',
 'молочница',
 'спасибо',
 'зарание',
 'ребенку',
 'поставили',
 'диагноз',
 'рефлюкс',
 'степени',
 'кровотечение',
 'слабость',
 'грибок',
 'коже',
 'консультация',
 'лечение',
 'коллег',
 'врач',
 'отоларинголог',
 'точнее',
 'дермокосметолог',
 'верно',
 'перезвоните',
 'пожалуйста',
 'телефону',
 'тест',
 'реагировать',
 'нужна',
 'консультация',
 'гинеколога',
 'прям',
 'сейчасочень',
 'важно',
 'благодарюа',
 'консультация',
 'уролога',
 'ребенка',
 'руке',
 'появились',
 'неизвестного',
 'происхождения',
 'раны',
 'требуется',
 'консультация',
 'спасибо',
 'последнее',
 'время',
 'стала',
 'страдать',
 'бессонницей',
 'совершенно',
 'хочу',
 'спатьначала',
 'принимать',
 'сна

In [ ]:
len(list_of_words)

373511

In [ ]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

47407

In [ ]:
# join the words into a string
comments['clean_joined'] = comments['clean'].apply(lambda x: " ".join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
comments.head()

,comment,doctor_id,clean,clean_joined
1,Прием к врачу Попович Срок беременности 15 нед...,36,"[прием, врачу, попович, срок, беременности, не...",прием врачу попович срок беременности недель з...
2,здраствуйте хотелабы у вас проконсультироваца ...,36,"[здраствуйте, хотелабы, проблема, большого, ха...",здраствуйте хотелабы проблема большого характе...
3,Ребенку поставили диагноз рефлюкс 23 степени,65,"[ребенку, поставили, диагноз, рефлюкс, степени]",ребенку поставили диагноз рефлюкс степени
4,Кровотечение слабость,152,"[кровотечение, слабость]",кровотечение слабость
5,Грибок на коже консультация и лечение,243,"[грибок, коже, консультация, лечение]",грибок коже консультация лечение


In [ ]:
# split data into test and train 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(comments.clean_joined, comments.doctor_id, test_size = 0.2)

In [ ]:
from nltk import word_tokenize

In [ ]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
len(train_sequences)

49678

In [ ]:
len(test_sequences)

12420

In [ ]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = 4405, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 4405, truncating = 'post') 

In [ ]:
for i,doc in enumerate(padded_train[:2]):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [30 14  5 ...  0  0  0]
The padded encoding for document 2  is :  [37 84  0 ...  0  0  0]


In [ ]:
padded_train[6]

array([ 144,  395, 3906, ...,    0,    0,    0], dtype=int32)

In [ ]:
# Sequential Model
model = Sequential()

# embeddidng layer
model.add(Embedding(total_words, output_dim = 128))
# model.add(Embedding(total_words, output_dim = 240))


# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         6068096   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 6,364,289
Trainable params: 6,364,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
total_words

47407

In [ ]:
y_train = np.asarray(y_train)

In [ ]:
padded_train

array([[   30,    14,     5, ...,     0,     0,     0],
       [   37,    84,     0, ...,     0,     0,     0],
       [16497, 11158,     0, ...,     0,     0,     0],
       ...,
       [ 1508,   904,   383, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    7,     1,   203, ...,     0,     0,     0]], dtype=int32)

In [1]:
y_train = y_train.astype(np.float32)
y_train

NameError: ignored

In [ ]:
# train the model
model.fit(padded_train, y_train, batch_size = 64, validation_split = 0.1, epochs = 2)

Epoch 1/2


UnimplementedError: ignored